In [ ]:
# Install medcat
! pip install medcat~=1.12.1
# Get the spacy model
! python -m spacy download en_core_web_md
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 549.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.met

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Restart the runtime if on colab, sometimes necessary after installing models**

In [ ]:
! python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

from matplotlib import pyplot as plt
from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.config import Config
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT
from medcat.config_meta_cat import ConfigMetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE
from medcat.tokenizers.meta_cat_tokenizers import TokenizerWrapperBERT
from tokenizers import ByteLevelBPETokenizer

/usr/local/lib/python3.10/dist-packages/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
DATA_DIR = "./data_p4.2/"
! DATA_DIR="./data_p4.2/"
vocab_path = DATA_DIR + "vocab.dat"
cdb_path = DATA_DIR + "cdb-medmen-v1.dat"

In [ ]:
# Download the models and required data
!wget -N https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/MedCAT_Export.json -P $DATA_DIR
# You can also use the models created in Part 4.1 of the Tutorial
!wget -N https://cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com/medcat-example-models/mc_status.zip -P $DATA_DIR

# Get MedCAT models components (Alternatively you can use a previously created MedCAT model packs)
!wget -N https://cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com/medcat-example-models/vocab.dat -P $DATA_DIR
!wget -N https://cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com/medcat-example-models/cdb-medmen-v1.dat -P $DATA_DIR

--2024-08-16 08:23:54--  https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/MedCAT_Export.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272538 (266K) [text/plain]
Saving to: ‘./data_p4.2/MedCAT_Export.json’

MedCAT_Export.json  100%[===================>] 266.15K  --.-KB/s    in 0.04s   

Last-modified header missing -- time-stamps turned off.
2024-08-16 08:23:54 (6.82 MB/s) - ‘./data_p4.2/MedCAT_Export.json’ saved [272538/272538]

--2024-08-16 08:23:54--  https://cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com/medcat-example-models/mc_status.zip
Resolving cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com (cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com)... 52.95.150.170, 3.5.246.110,

## MedCATtrainer Export

In [ ]:
#@title
data = json.load(open(DATA_DIR + "MedCAT_Export.json"))

In [ ]:
#@title
print(data.keys())

dict_keys(['projects'])


In [ ]:
#@title
data['projects'][0].keys()

dict_keys(['name', 'id', 'cuis', 'tuis', 'documents'])

In [ ]:
#@title
data['projects'][0]['documents'][0].keys()

dict_keys(['id', 'name', 'text', 'last_modified', 'annotations'])

In [ ]:
#@title
data['projects'][0]['documents'][0]['annotations'][0].keys()

dict_keys(['id', 'user', 'cui', 'value', 'start', 'end', 'validated', 'correct', 'deleted', 'alternative', 'killed', 'last_modified', 'manually_created', 'acc', 'meta_anns'])

In [ ]:
#@title
data['projects'][0]['documents'][0]['annotations'][0]['meta_anns'][0].keys()

dict_keys(['name', 'value', 'acc', 'validated'])

##Fine-tuning the NER+L model

First we load the existing MedCAT models that we will fine-tune.



In [ ]:
# Create and load the CDB (Concept Database)
cdb = CDB.load(cdb_path)
# NOTE: CDBs saved prior to medcat 1.10 will also load a config
#       as part of the CDB, but below we'll be changing that anyway

# Create and load the Vocabulary
vocab = Vocab.load(vocab_path)

# Setup config
config = Config()
config.general['spacy_model'] = 'en_core_web_md'

# Create CAT - the main class from medcat used for concept annotation
cat = CAT(cdb=cdb, config=config, vocab=vocab)

To fine-tune the existing models we use the `train_supervised_from_json()` method from MedCAT. The method has the following options:


---



`data_path` - Path to the JSON file exported from MedCATtrainer

`reset_cui_count` - Each cui has an internal counter that is used for weight decay, this will reset it.

`nepochs` - Numeber of epochs

`reset_cui_count` - Used for training with weight_decay (annealing). Each concept has a count that is there from the beginning of the CDB, that count is used for annealing. Resetting the count will significantly increase the training impact. This will reset the count only for concepts that exist in the the training data.

`print_stats` - Print statistics during training (F1/P/R)

`test_set` - Provide another file for testing

`use_filters` - During the training do you want to set the cui/tui filters from the medcattrainer project configuration.

`never_terminate` - If True no termination of concepts will be applied

`terminate_last` - If true, concept termination will be done after all training.

`use_groups` - If True concepts that have groups will be combined and stats will be reported on groups.

`use_overlaps` - Allow overlapping entites, nearly always False as it is very difficult to annotate overlapping entites.

`use_cui_doc_limit` - If True the metrics for a CUI will be only calculated if that CUI appears in a document.

`train_from_false_positives` - If True it will use false positive examples detected by medcat and train from them as negative examples.



In [ ]:
cat.train_supervised_from_json(data_path=DATA_DIR + "MedCAT_Export.json",
                               nepochs=1,
                               reset_cui_count=False,
                               print_stats=True,
                               use_filters=True)

INFO:medcat:Running without a test set, or train==test


Stats project: 0%| | 0/1 [00:00<?, ?it/s]

Stats document: 0%| | 0/27 [00:00<?, ?it/s]

Epoch: 0, Prec: 0.7948717948717948, Rec: 0.7828282828282829, F1: 0.7888040712468194

Docs with false positives: 2124; 1383; 516; 1734; 96; 1577; 1881; 1687; 899; 688

Docs with false negatives: 1383; 1734; 73; 96; 1577; 1881; 1070; 688; 1737; 716



False Positives

Diabetes - C0011847 - 18
Obesity - C0028754 - 5
Hypertensive disease - C0020538 - 5
nervous system disorder - C0027765 - 4
Disease - C0012634 - 3
Alzheimer's Disease - C0002395 - 3
Anxiety Disorders - C0003469 - 3
Kidney Diseases - C0022658 - 3
Coronary Arteriosclerosis - C0010054 - 2
Myocardial Diseases, Secondary - C0036529 - 2


False Negatives

disorder lesions skin - C0037284 - 13
Diabetes Mellitus, Non-Insulin-Dependent - C0011860 - 9
Diabetes Mellitus - C0011849 - 7
Lymphadenopathy - C0497156 - 6
Heart Diseases - C0018799 - 4
Arthritis - C0003864 - 3
Pseudomonas aeruginosa infection - C0854135 - 3
Pulmonary Embolism - C0034065 - 3
Renal Insufficiency - C1565489 - 3
Coronary heart disease - C0010068 - 2


True Positiv

Epoch: 0%| | 0/1 [00:00<?, ?it/s]

...

Document: 0%| | 0/27 [00:00<?, ?it/s]

Stats project: 0%| | 0/1 [00:00<?, ?it/s]

Stats document: 0%| | 0/27 [00:00<?, ?it/s]

Epoch: 1, Prec: 0.8439024390243902, Rec: 0.8737373737373737, F1: 0.858560794044665

Docs with false positives: 2124; 1383; 516; 96; 1577; 1881; 1687; 688; 1737; 716

Docs with false negatives: 1881; 1605; 1070; 1694; 611; 1737; 1377; 1383; 716; 323



False Positives

Diabetes - C0011847 - 15
Hypertensive disease - C0020538 - 5
Diabetes Mellitus - C0011849 - 4
Disease - C0012634 - 4
Cognition Disorders - C0009241 - 3
Anxiety Disorders - C0003469 - 3
Coronary Arteriosclerosis - C0010054 - 2
Myocardial Diseases, Secondary - C0036529 - 2
Diabetes Mellitus, Non-Insulin-Dependent - C0011860 - 2
disorder lesions skin - C0037284 - 2


False Negatives

Diabetes Mellitus, Non-Insulin-Dependent - C0011860 - 9
Diabetes Mellitus - C0011849 - 7
Pseudomonas aeruginosa infection - C0854135 - 3
Pulmonary Embolism - C0034065 - 3
Coronary heart disease - C0010068 - 2
Arthritis - C0003864 - 2
Syndrome - C0039082 - 2
disorder lesions skin - C0037284 - 2
Ulcer - C0041582 - 2
Diabetes Mellitus, Insulin-Depe

({'C0011847': 15,
 'C0020538': 5,
 'C0011849': 4,
 'C0012634': 4,
 'C0009241': 3,
 'C0003469': 3,
 'C0010054': 2,
 'C0036529': 2,
 'C0011860': 2,
 'C0037284': 2,
 'C0233794': 2,
 'C0276289': 2,
 'C0349782': 2,
 'C0029456': 2,
 'C0028754': 1,
 'C0038443': 1,
 'C0002792': 1,
 'C0009319': 1,
 'C0018939': 1,
 'C0339573': 1,
 'C1704436': 1,
 'C0038454': 1,
 'C0018801': 1,
 'C0023351': 1,
 'C0029408': 1,
 'C0030567': 1,
 'C0031099': 1,
 'C0519066': 1},
 {'C0011860': 9,
 'C0011849': 7,
 'C0854135': 3,
 'C0034065': 3,
 'C0010068': 2,
 'C0003864': 2,
 'C0039082': 2,
 'C0037284': 2,
 'C0041582': 2,
 'C0011854': 2,
 'C0018939': 1,
 'C0238792': 1,
 'C0008679': 1,
 'C1301700': 1,
 'C0021167': 1,
 'C0009324': 1,
 'C0018799': 1,
 'C0002871': 1,
 'C3844825': 1,
 'C0018889': 1,
 'C0033377': 1,
 'C0042769': 1,
 'C0263746': 1,
 'C0206172': 1,
 'C0021400': 1,
 'C0085762': 1},
 {'C0020538': 43,
 'C0037284': 19,
 'C0020473': 12,
 'C0010054': 10,
 'C0497156': 10,
 'C0011570': 10,
 'C0041834': 9,
 'C0018802':

In [ ]:
# If we want to know the F1, P, R for each cui, we can call the stats method
data = json.load(open(DATA_DIR + "MedCAT_Export.json"))
fps, fns, tps, cui_prec, cui_rec, cui_f1, cui_counts, examples = cat._print_stats(data, extra_cui_filter=True)

Stats project: 0%| | 0/1 [00:00<?, ?it/s]

Stats document: 0%| | 0/27 [00:00<?, ?it/s]

Epoch: 0, Prec: 0.07726965532777652, Rec: 0.8511166253101737, F1: 0.1416769929781082

Docs with false positives: 2124; 1383; 1734; 516; 73; 96; 1577; 1881; 1070; 1687

Docs with false negatives: 1383; 73; 96; 1577; 1881; 1070; 688; 1737; 716; 496



False Positives

Patients - C0030705 - 200
Normal - C0205307 - 67
Medical History - C0262926 - 57
Daily - C0332173 - 46
year - C0439234 - 41
Bilateral - C0238767 - 40
Right - C0205090 - 33
Pain - C0030193 - 28
Physical Examination - C0031809 - 27
Lung - C0024109 - 25


False Negatives

Diabetes Mellitus, Non-Insulin-Dependent - C0011860 - 9
Diabetes Mellitus - C0011849 - 7
disorder lesions skin - C0037284 - 4
Pseudomonas aeruginosa infection - C0854135 - 3
Pulmonary Embolism - C0034065 - 3
Coronary heart disease - C0010068 - 2
Arthritis - C0003864 - 2
C1561826 - C1561826 - 2
Syndrome - C0039082 - 2
Ulcer - C0041582 - 2


True Positives

Hypertensive disease - C0020538 - 43
disorder lesions skin - C0037284 - 17
Hyperlipidemia - C0020473 - 12

#### K-fold metrics
K-fold cross-validation offers a more robust evaluation of your model's performance by dividing your dataset into k subsets, or folds.
Unlike a single evaluation on the entire dataset (like `cat._print_stats`), the k-fold approach ensures that every data point is used for both training and validation, thereby reducing the risk of bias and providing a more reliable estimate of the model's generalization capabilities.
This method is particularly beneficial for assessing the fine-tuned performance of your model on specific datasets, as it accounts for variability and offers a comprehensive understanding of how the model might perform on unseen data.

In [ ]:
# you need to import the module to use it
from medcat.stats.kfold import get_k_fold_stats
fps, fns, tps, cui_prec, cui_rec, cui_f1, cui_counts, examples = get_k_fold_stats(cat, data)

In [ ]:
# Now we can check F1/P/R for a cui using
cui = "C0020538" # Hypertension
print(cui_f1[cui], cui_prec[cui], cui_rec[cui])

0.9662921348314606 0.9347826086956522 1.0


In [ ]:
# inspect fn, fp examples for specific CUIs of interest...
examples['fn']['C0010068']

[{'text': ' appendectomy, oophorectomy.,FAMILY HISTORY: , Positive for coronary artery disease in her father and brother in their 40s.,SOCIAL HISTORY: , S',
 'cui': 'C0010068',
 'source value': 'coronary artery disease',
 'acc': 1,
 'project index': 0,
 'document inedex': 1},
 {'text': 'on fraction of 20%-25% in December 2005, COPD, mild diffuse coronary artery disease, and renal insufficiency.,ALLERGIES:, NO KNOWN DRUG ALLERGI',
 'cui': 'C0010068',
 'source value': 'coronary artery disease',
 'acc': 1,
 'project index': 0,
 'document inedex': 25}]

In [ ]:
# annotations inside first projects...
[anno for d in data['projects'][0]['documents'] for anno in d['annotations']][0:5]

[{'id': 45580,
 'user': 'wish',
 'cui': 'C0017168',
 'value': 'gastroesophageal reflux',
 'start': 332,
 'end': 355,
 'validated': True,
 'correct': True,
 'deleted': False,
 'alternative': False,
 'killed': False,
 'last_modified': '2020-04-01 22:06:34.303633+00:00',
 'manually_created': False,
 'acc': 1.0,
 'meta_anns': [{'name': 'Status',
 'value': 'Other',
 'acc': 1.0,
 'validated': True}]},
 {'id': 45581,
 'user': 'wish',
 'cui': 'C0020538',
 'value': 'hypertension',
 'start': 255,
 'end': 267,
 'validated': True,
 'correct': True,
 'deleted': False,
 'alternative': False,
 'killed': False,
 'last_modified': '2020-04-01 22:06:30.394941+00:00',
 'manually_created': False,
 'acc': 1.0,
 'meta_anns': [{'name': 'Status',
 'value': 'Confirmed',
 'acc': 1.0,
 'validated': True}]},
 {'id': 45582,
 'user': 'wish',
 'cui': 'C0012634',
 'value': 'disorder',
 'start': 356,
 'end': 364,
 'validated': True,
 'correct': False,
 'deleted': False,
 'alternative': False,
 'killed': True,
 'last_mo

##MetaAnnotations

During the annotation process we have created only one meta-annotation called "Status". Here we are going to train a BiLSTM to detect that meta-annotation.

### MetaCAT

The class we are going to use to train meta-annotations is called MetaCAT. As input it takes:

`tokenizer` - BBPE tokenizer from [huggingface](https://github.com/huggingface/tokenizers) and BERT tokenizer

`embeddings` - The precalculated embeddings for the tokens produced by the tokenizer. Can be anything from Word2Vec to BERT. This is a numpy matrix, or python list of embeddings (only for Bi-LSTM).

`cntx_left` - Size of context from the left side of the entity that will be taken into account.

`cntx_right` - Size of context from the right side of the entity that will be taken into account.

`save_dir` - Where do we want to save the trained models.

`pad_id` - Padding index in the embeddings matrix.

`device` - On which device to run this `cpu` or `cuda`

#### For BERT
You can choose to train `BERT` or `Bi-LSTM` model for MetaCAT. <br>
In this tutorial, we'll demonstrate:
*   continuing training for a previously trained `Bi-LSTM` model (on NHS data)
* training `BERT` without loading a previously trained model (on NHS data)

In [ ]:
# training BERT without loading previous model pack

config_metacat = ConfigMetaCAT()
config_metacat.model['model_name'] = 'bert'

# add category name
config_metacat.general['category_name'] = 'Status'
# add number of classes
config_metacat.model['nclasses'] = 2

tokenizer = TokenizerWrapperBERT.load("", config_metacat.model['model_variant'])

If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`. Loading from library for model variant: bert-base-uncased


In [ ]:
mc = MetaCAT(tokenizer=tokenizer, embeddings=None, config=config_metacat)

Input size for bert-base-uncased model should be 768, provided input size is 300 Input size changed to 768
INFO:medcat.meta_cat:BERT model used for classification


##### MetaCAT configuration

For a full list of all the configurable parameter. Follow this [link](https://github.com/CogStack/MedCAT/blob/master/medcat/config_meta_cat.py).

Some noteable parameters:

`category_name` - What is the name of this meta-annotation (same as the name in the MedCATtrainer)

`model_name` - `bert` or `lstm`

`lr` - Learning rate

`test_size` - Proportion of the test set

`bach_size` - Batch size

`model_freeze_layers` - Only for BERT: Freeze BERT's layers or train using LoRA

`metric` - What metric should be used for choosing the best model.

`nepochs` - Number of epochs to run for

`lowercase` - Do you want to lowercase the text

`class_weights` - Pytorch parameter for unbalanced classes

`compute_class_weights` - Do you want the classes weights to be computed automatically based on the data

`ignore_cpos` - The position of the entity will be ignored, do not use this.

`auto_save_model` - This will autosave the top performing epoch during the training process

In [ ]:
# Example of how to change parameters
mc.config.model['input_size'] = 768
mc.config.model['hidden_size'] = 300

mc.config.train.lr = 5e-4
mc.config.model.dropout = 0.2

mc.config.model['model_freeze_layers'] = False
mc.config.train['nepochs'] = 55
# mc.config.train.metric['base'] = 'macro avg'
mc.config.train['auto_save_model'] = True

In [ ]:
import logging
logging.basicConfig(level=logging.INFO,force=True)

##### Train MetaCAT
To run the training we use the `train_from_json` method that allows us to specify:

`json_path`: Path to a MedCATtrainer export containing the meta_annotations we want to train for.


`save_dir_path`: (optional, defaults to `None`): In case we have auto_save_model (meaning during the training the best model will be saved) we need to set a save path.


In [ ]:
mc.train_from_json(json_path=DATA_DIR+"MedCAT_Export.json", save_dir_path='status_bert')
# Saving the model this way will only save the model epoch with the best performance

INFO:medcat.utils.meta_cat.data_utils:Updated label_data: {0: 128, 1: 128}
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:medcat.utils.meta_cat.ml_utils:Total steps for optimizer: 200
INFO:medcat.utils.meta_cat.ml_utils:Epoch: 0 ************************************************** Train
INFO:medcat.utils.meta_cat.ml_utils:              precision    recall  f1-score   support

           0       0.69      0.64      0.66       251
           1       0.31      0.36      0.33       114

    accuracy                           0.55       365
   macro avg       0.50      0.50      0.50       365
weighted avg       0.57      0.55      0.56       365

INFO:medcat.utils.meta_cat.ml_utils:Epoch: 0 ***************************************

{'confusion_matrix': array([[0.85185185, 0.14814815],
        [0.14285714, 0.85714286]]),
 'report': {'0': {'precision': 0.92,
   'recall': 0.8518518518518519,
   'f1-score': 0.8846153846153846,
   'support': 27.0},
  '1': {'precision': 0.75,
   'recall': 0.8571428571428571,
   'f1-score': 0.7999999999999999,
   'support': 14.0},
  'accuracy': 0.8536585365853658,
  'macro avg': {'precision': 0.835,
   'recall': 0.8544973544973544,
   'f1-score': 0.8423076923076922,
   'support': 41.0},
  'weighted avg': {'precision': 0.8619512195121952,
   'recall': 0.8536585365853658,
   'f1-score': 0.8557223264540337,
   'support': 41.0}},
 'report_train': {'0': {'precision': 0.9626556016597511,
   'recall': 0.9243027888446215,
   'f1-score': 0.9430894308943089,
   'support': 251.0},
  '1': {'precision': 0.8467741935483871,
   'recall': 0.9210526315789473,
   'f1-score': 0.8823529411764707,
   'support': 114.0},
  'accuracy': 0.9232876712328767,
  'macro avg': {'precision': 0.9047148976040691,
   're

#### For Bi-LSTM

In [ ]:
!unzip $DATA_DIR/mc_status.zip

Archive:  ./data_p4.2//mc_status.zip
   creating: Status/
  inflating: Status/model.dat        
  inflating: Status/config.json      
  inflating: Status/bbpe-vocab.json  
  inflating: Status/bbpe-merges.txt  


In [ ]:
# Get the required tokenizer (note that we have already downloaded the required models)
mc = MetaCAT.load('Status')

##### MetaCAT configuration

For a full list of all the configurable parameter. Follow this [link](https://github.com/CogStack/MedCAT/blob/master/medcat/config_meta_cat.py).

Some noteable parameters:

`category_name` - What is the name of this meta-annotation (same as the name in the MedCATtrainer)

`model_name` - `bert` or `lstm`

`lr` - Learning rate

`test_size` - Proportion of the test set

`bach_size` - Batch size

`model_freeze_layers` - Only for BERT: Freeze BERT's layers or train using LoRA

`metric` - What metric should be used for choosing the best model.

`nepochs` - Number of epochs to run for

`lowercase` - Do you want to lowercase the text

`class_weights` - Pytorch parameter for unbalanced classes

`compute_class_weights` - Do you want the classes weights to be computed automatically based on the data

`ignore_cpos` - The position of the entity will be ignored, do not use this.

`auto_save_model` - This will autosave the top performing epoch during the training process

In [ ]:
# Example of how to change parameters
mc.config.model['input_size'] = 768
mc.config.model['hidden_size'] = 300
mc.config.general['category_value2id'] = {'Other':1,'Confirmed':0}

mc.config.train['test_size'] = 0.2

# mc.config.train['compute_class_weights'] = True

mc.config.model['nclasses'] = 2
mc.config.train['nepochs'] = 55
mc.config.train['auto_save_model'] = True

##### Train MetaCAT
To run the training we use the `train_from_json` method that allows us to specify:

`json_path`: Path to a MedCATtrainer export containing the meta_annotations we want to train for.


`save_dir_path`: (optional, defaults to `None`): In case we have auto_save_model (meaning during the training the best model will be saved) we need to set a save path.


In [ ]:
import logging
logging.basicConfig(level=logging.INFO,force=True)

In [ ]:
mc.train_from_json(json_path= DATA_DIR+"MedCAT_Export.json", save_dir_path='status_lstm')
# Saving the model this way will only save the model epoch with the best performance

INFO:medcat.utils.meta_cat.data_utils:Updated label_data: {1: 128, 0: 128}
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:medcat.utils.meta_cat.ml_utils:Total steps for optimizer: 178
INFO:medcat.utils.meta_cat.ml_utils:Epoch: 0 ************************************************** Train
INFO:medcat.utils.meta_cat.ml_utils:              precision    recall  f1-score   support

           0       0.89      0.93      0.91       222
           1       0.83      0.75      0.79       102

    accuracy                           0.87       324
   macro avg       0.86      0.84      0.85       324
weighted avg       0.87      0.87      0.87       324

INFO:medcat.utils.meta_cat.ml_utils:Epoch: 0 ***************************************

{'confusion_matrix': array([[0.85714286, 0.14285714],
        [0.15384615, 0.84615385]]),
 'report': {'0': {'precision': 0.9230769230769231,
   'recall': 0.8571428571428571,
   'f1-score': 0.888888888888889,
   'support': 56.0},
  '1': {'precision': 0.7333333333333333,
   'recall': 0.8461538461538461,
   'f1-score': 0.7857142857142856,
   'support': 26.0},
  'accuracy': 0.8536585365853658,
  'macro avg': {'precision': 0.8282051282051281,
   'recall': 0.8516483516483516,
   'f1-score': 0.8373015873015872,
   'support': 82.0},
  'weighted avg': {'precision': 0.8629143214509067,
   'recall': 0.8536585365853658,
   'f1-score': 0.8561749903213317,
   'support': 82.0}},
 'report_train': {'0': {'precision': 0.9641255605381166,
   'recall': 0.9684684684684685,
   'f1-score': 0.9662921348314607,
   'support': 222.0},
  '1': {'precision': 0.9306930693069307,
   'recall': 0.9215686274509803,
   'f1-score': 0.9261083743842364,
   'support': 102.0},
  'accuracy': 0.9537037037037037,
  'macro avg': 

We can now save the models using the `save` function. It has only one argument `full_save` if `True` it will also save the embeddings and tokenizers (note that this is slightly redundant, as we no training was done on the embeddings/tokenizers).

In [ ]:
# Alternative way to save
mc.save("alternative_status_metamodel")

## Test of the whole pipeline

In [ ]:
# Set filter
from medcat.config import Config
config = Config()
tui_filter = ['T047'] # Detect only diseases
cui_filters = set()
for tui in tui_filter:
    cui_filters.update(cdb.addl_info['type_id2cuis'][tui])
cdb.config.linking['filters']['cuis'] = cui_filters

# To add the meta-annotation model to the medcat pipeline
cat = CAT(cdb=cdb, config=config, vocab=vocab, meta_cats=[mc])

In [ ]:
"C0035078" in cui_filters

True

In [ ]:
text = "John Doe has no history of diabetes. He has epilepsy"
doc = cat(text)

In [ ]:
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: history of diabetes
Meta Annotations: {'Status': {'value': 'Other', 'confidence': 0.9942723512649536, 'name': 'Status'}}


Entity: epilepsy
Meta Annotations: {'Status': {'value': 'Confirmed', 'confidence': 0.9394962787628174, 'name': 'Status'}}




Notice how the medcat metamodel labels John Doe with entity: "diabetes",  __"Other"__.

Whilst "epilepsy" is labelled as __"Confirmed"__.

This is extremely useful when conducting a context-based extract of concepts from text.

End of Tutorial